In [ ]:
import face_recognition
import glob
import numpy as np
import pandas as pd
import cv2
import os

In [ ]:
#load features first
feature_dir = ''


In [ ]:
def change(s):
    return eval(s)

In [ ]:
#read members feature

def read_members_frame(features_dir):
    all_features = pd.read_csv(features_dir, index_col=0, encoding = 'utf-8')  
    return all_features

In [ ]:
#input img directory, feature directory
#output message
def create_member(img_dir, features_dir):
    img_list = glob.glob(img_dir+'/*')
    
    member_name = os.path.split(img_list[0])[-1].split(".")[0].split("_")[0]
    dummy_coding = [0]*128
    member_features = []
    features_list = read_members_frame(features_dir)
    
    df_index=features_list.columns
    if(df_index.contains(member_name)):
        message = 'member already exists'
        return message
    
    for (i,f) in enumerate(img_list):
        mem_img = face_recognition.load_image_file(f)
        face_encoding = face_recognition.face_encodings(mem_img)
        if face_encoding:
            member_features.append(list(face_encoding[0]))
        else:
            member_features.append(list(dummy_coding))

    #create new member feature
    df_cols = features_list.shape[1]
    #insert new member
    features_list.insert(df_cols, member_name, member_features)

    features_list.to_csv(features_dir, encoding='utf-8')
    message = 'new member created'

    return message

In [ ]:
#input features list
def resume_all_features(features_list):
    df_cols = features_list.shape[1]
    #change string back to float list
    for i in range(1,df_cols):
        current_col = features_list.columns[i]
        features_list[current_col] = features_list[current_col].apply(change)
    
    applied = True
    #one column can only be apply once
    return applied

In [ ]:
#input cv image encoding and single member features
def find_match(cv_encoding, feature):
    results = face_recognition.compare_faces(cv_encoding, feature, tolerance = 0.3)     

    check_true = results.count(True)
    #check if feature matches
    if (check_true==5):
        found = True
    else:
        found = False
    
    return found

In [ ]:
#input cv image and features list
def member_login(cv_img,features_list):
   
    df_cols = features_list.shape[1]
    cv_feature = face_recognition.face_encodings(cv_img)
  
    result_name = 'not found'
    if not cv_feature:
        return result_name
    cv_encoding = cv_feature[0]
  
    for i in range(1,df_cols):
        current_col = features_list.columns[i]
        #check current column
        found = find_match(features_list[current_col].tolist(), cv_encoding)
    #return current column name if found
    if(found):
        result_name = current_col
        break

    return result_name

In [ ]:
#註冊:
#給成員影像位置
#create_member
#print(message)

In [ ]:
#登入
#先讀所有會員特徵向量
#只做一次 resume_all_features
#member login output string:result and blooing:found
#得到輸出後決定下一步驟